In [1]:
import pandas as pd
import keras
import keras_metrics
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

x_train = pd.read_csv('x_train.csv')
y_train = pd.read_csv('y_train.csv')
x_test = pd.read_csv('x_test.csv')
y_test = pd.read_csv('y_test.csv')

In [2]:
# Training the tokenizer and using that tokenizer to convert a list of tokens to a list of numbers
# Instantiating and fit the tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train['cleanned_text'])

# Use the tokenizer to transform the text messages in the training and test sets
x_train_seq = tokenizer.texts_to_sequences(x_train['cleanned_text'])
x_test_seq = tokenizer.texts_to_sequences(x_test['cleanned_text'])

In [3]:
# Padding each sequesne to the same length, '50' here: Padding the one less than 50 to 50 with 0s
# and truncating the longer ones down to 50
x_train_seq_padded = pad_sequences(x_train_seq, 50)
x_test_seq_padded = pad_sequences(x_test_seq, 50)

In [4]:
# Constructing a simple RNN model
model = Sequential()
model.add(Embedding(len(tokenizer.index_word)+1, 32))
model.add(LSTM(32, dropout = 0, recurrent_dropout = 0))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          263904    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 273,313
Trainable params: 273,313
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Compiling the model
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy', keras_metrics.precision(), keras_metrics.recall()])

In [6]:
import warnings
warnings.filterwarnings("ignore")
# Fit the model
history = model.fit(x_train_seq_padded, y_train,
                    batch_size = 32, epochs = 5,
                    validation_data = (x_test_seq_padded, y_test))

Epoch 1/5
140/140 [==============================] - 14s 58ms/step - loss: 0.4484 - accuracy: 0.8483 - precision: 0.2199 - recall: 0.1630 - val_loss: 0.0732 - val_accuracy: 0.9794 - val_precision: 0.9977 - val_recall: 0.8545
Epoch 2/5
140/140 [==============================] - 5s 32ms/step - loss: 0.0347 - accuracy: 0.9932 - precision: 0.9975 - recall: 0.9596 - val_loss: 0.0649 - val_accuracy: 0.9821 - val_precision: 0.9993 - val_recall: 0.8646
Epoch 3/5
140/140 [==============================] - 4s 31ms/step - loss: 0.0099 - accuracy: 0.9982 - precision: 0.9995 - recall: 0.9908 - val_loss: 0.0512 - val_accuracy: 0.9883 - val_precision: 0.9994 - val_recall: 0.9408
Epoch 4/5
140/140 [==============================] - 4s 30ms/step - loss: 0.0035 - accuracy: 0.9994 - precision: 1.0000 - recall: 0.9971 - val_loss: 0.0673 - val_accuracy: 0.9865 - val_precision: 0.9993 - val_recall: 0.9301
Epoch 5/5
140/140 [==============================] - 4s 31ms/step - loss: 7.2318e-04 - accuracy: 0.9999

In [7]:
# TFIDF: Accuracy: 0.9695 / Precision: 1.0 / Recall: 0.7888
# The RNN model outperformed Word_2_Vec and Doc_2_Vec in all the 3 metrics, and the TFIDF model in accuracy and recall 
# metrics while the precision is almost as good. The improvement in recall though is significant.